In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import pandas as pd
from stratosphere import Stratosphere
from stratosphere.utils.inspect_flows import InspectFlows
from stratosphere import options
from stratosphere.services.extractor import Extractor
from stratosphere.storage.models import Flow

In [4]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

[[Home](/)]
[[Webapps](/jupyter/voila/tree/webapps)]

# Google searches



In [5]:
kb = Stratosphere(options.get("db.url_kb"))


In [6]:
def get_google_searches():
    
    q = """
        SELECT
            entity_id,
            json_extract(data, '$.q') as query
        FROM entities
        WHERE type = "search_string"
    """
    df = kb.db.pandas(q)
    
    return df.set_index('query').to_dict(orient='index')

#get_google_searches()

In [7]:
def get_google_results(entity_id):
    
    q = f"""
    SELECT
        e.entity_id,
        e.type,
        r.src,
        json_extract(e.data, '$.text') as text,
        json_extract(e.data, '$.url') as url
    FROM entities e
    LEFT JOIN relationships r ON e.entity_id = r.dst
    WHERE
        e.type = "search_result" AND
        r.src = "{entity_id}"
    """
    df = kb.db.pandas(q)
    return df

#get_google_results('1d9b08d14cbfb686c2e1e760b017d384')



In [8]:
# Imports
import pandas as pd
import textwrap
from datetime import datetime
from ipywidgets import HTML
from ipywidgets import GridspecLayout, Layout, Button, Textarea
import ipywidgets as widgets

import ipywidgets as widgets

options = get_google_searches()
w_searches = widgets.Select(options=options, value=list(options.values())[0], rows=20, disabled=False, layout=Layout(width="99%", height='180px'))



In [9]:
w_out = widgets.Output()


In [10]:
from IPython.display import HTML

def render_img_url(url):
    return '<img src="'+ url + '" style=max-height:124px;"/>'

def render_href(href):    
    return f'<a href={href}>{href}</a>'


from markdown import markdown

def w_searches_onchange(change):
    
    mks = []
    
    if change['name'] == 'value':
        w_out.clear_output()
        with w_out:
            df = get_google_results(w_searches.value['entity_id'])
            
            for idx, row in df.iterrows():
                text = row.text if row.text is not None else row.url
                
                from urllib.parse import urlparse
                domain = urlparse(row.url).netloc
                
                mks.append(f"{idx+1}. [[e](/jupyter/voila/render/webapps/entity-overview.ipynb?entity_id={row.entity_id})] [{str(text)}]({row.url}) - **{domain}**")
            
            
            cols = ["entity_id", "text", "url"]
            df = df[cols]
#            df = df.style.set_properties( **{'width': '50px', })
#            html = HTML(df.to_html(escape=False , col_space='150px', render_links=True)) # formatters=dict(url=render_href)
 #           display(html)
    
            html = markdown('\n'.join(mks))
            html = html.replace('<ol>', '<ol style="list-style-position: inside;">')
            html = html.replace('<a ', '<a target="_blank" ')
        
            display(HTML(html))
            
w_searches.observe(w_searches_onchange)

w_searches_onchange({'name':'value'})

w_searches


Select(layout=Layout(height='180px', width='99%'), options={'bellingcat tools': {'entity_id': '7b2c33c94f2903d…

In [11]:
w_out

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', '…